This notebook is inspired from [@garethjns ideas & work](https://www.kaggle.com/garethjns/convolutional-deep-q-learner) and [Alexander Van de Kleut's GitHub](https://alexandervandekleut.github.io/) (especially [this article](https://alexandervandekleut.github.io/deep-q-learning/))

We first install the required components

In [ ]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.7 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.7.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install . 

# Install Gym
!pip install gym

from IPython.display import clear_output
clear_output()

Below are imported the useful packages

In [ ]:
import gfootball
import gym
import numpy as np
import tensorflow as tf
import random
from collections import deque

Q-Learning associates the "goodness" of picking an action a_t in a state s_t, while following a certain policy. That 's what we call the Q-Value. The policy is then created by selecting for each state the action maximizing the Q_value. [See here](https://alexandervandekleut.github.io/mdp/) and [here](https://alexandervandekleut.github.io/q-learning-numpy/)

We create a ReplayBuffer to remember state transitions. It helps the agent to perform better on old transitions. We sample them randomly to get a nice balance between ancient and new experiences.

In [ ]:
class ReplayBuffer:
    def __init__(self, size=1000000):
        self.memory = deque(maxlen=size)
        
    def remember(self, s_t, a_t, r_t, s_t_next, d_t):
        self.memory.append((s_t, a_t, r_t, s_t_next, d_t))
        
    def sample(self,batch_size):
        batch_size = min(batch_size, len(self.memory))
        return random.sample(self.memory,batch_size)

Our Agent will use the SMM representation of the Gfootball environment

From [Gfootball GitHub](https://github.com/google-research/football/blob/master/gfootball/doc/observation.md)
Simplified spacial (minimap) representation of a game state. It consists of several 72 * 96 planes of bytes, filled in with 0s except for:

1st plane: 255s represent positions of players on the left team

2nd plane: 255s represent positions of players on the right team

3rd plane: 255s represent position of a ball

4th plane: 255s represent position of an active player

As [@garethjns](https://www.kaggle.com/garethjns/convolutional-deep-q-learner) mentionned, Conv2D Layers will have more efficiency by treating only one plane. Thereby we need to create a SplitLayer

In [ ]:
class SplitLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        split0,split1,split2,split3 = tf.split(inputs,4,3)
        return [split0,split1,split2,split3]

To determinate Q values we use the neural network described below. Each output of the SplitLayer is going through a sequence of layers built by the build_conv_branch function. The results of each branch is concatened with the others one. After a couple of Dense layers the action chosen is given by the action_output layer.

In [ ]:
class ConvNN:

    def __init__(self,state_shape,num_actions):
        self.state_shape = state_shape
        self.num_actions = num_actions

    def _model_architecture(self):
    
        frames_input = tf.keras.layers.Input(shape=self.state_shape,name='input',batch_size=1)
        frames_split = SplitLayer()(frames_input)
        conv_branches = []
        for f,frame in enumerate(frames_split):
            conv_branches.append(self._build_conv_branch(frame,f))

        concat = tf.keras.layers.concatenate(conv_branches)
        
        fc0 = tf.keras.layers.Dense(units=8192, name='fc0', 
                                     activation='relu')(concat)
        fc1 = tf.keras.layers.Dense(units=2048, name='fc1', 
                                     activation='relu')(fc0)
        fc2 = tf.keras.layers.Dense(units=512, name='fc2', 
                                     activation='relu')(fc1)
        fc3 = tf.keras.layers.Dense(units=256, name='fc3', 
                                     activation='relu')(fc2)
        fc4 = tf.keras.layers.Dense(units=64, name='fc4', 
                                     activation='relu')(fc3)
        
        action_output = tf.keras.layers.Dense(units=self.num_actions, name='output',
                                               activation='relu')(fc4)
        return frames_input,action_output
        
    @staticmethod
    def _build_conv_branch(frame,number):
        conv1 = tf.keras.layers.Conv2D(16, kernel_size=(8, 8), strides=(4, 4),
                                        name='conv1_frame'+str(number), padding='same',
                                        activation='relu')(frame)
        mp1 = tf.keras.layers.MaxPooling2D(pool_size=2, name="mp1_frame"+str(number))(conv1)
        conv2 = tf.keras.layers.Conv2D(24, kernel_size=(4, 4), strides=(2, 2),
                                        name='conv2_frame'+str(number),padding='same',
                                        activation='relu')(mp1)
        mp2 = tf.keras.layers.MaxPooling2D(pool_size=2, name="mp2_frame"+str(number))(conv2)
        conv3 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                                        name='conv3_frame'+str(number),padding='same',
                                        activation='relu')(mp2)
        mp3 = tf.keras.layers.MaxPooling2D(pool_size=2, name="mp3_frame"+str(number))(conv3)
        conv4 = tf.keras.layers.Conv2D(64, kernel_size=(2, 2), strides=(1, 1),
                                        name='conv4_frame'+str(number), padding='same',
                                        activation='relu')(mp3)
        mp4 = tf.keras.layers.MaxPooling2D(pool_size=1, name="mp4_frame"+str(number))(conv4)

        flatten = tf.keras.layers.Flatten(name='flatten'+str(number))(mp4)

        return flatten
        
    def build(self):
        frames_input,action_output = self._model_architecture()
        model = tf.keras.Model(inputs=[frames_input], outputs=[action_output])
        return model

The agent starts by exploring the environment, picking random actions. The first Q-values are then determinated. As the epsilon is decaying, it get more interested in using his neural network to estimate the next action to choose. The update method maximizes the next Q-value,judge the current Q-value, calculate the loss function, and minimize it. [See here](https://alexandervandekleut.github.io/function-approximation/) and [here](https://alexandervandekleut.github.io/q-learning-tensorflow/)

In [ ]:
class Agent:
    def __init__(self, state_shape, num_actions,alpha, gamma, epsilon_i=1.0, epsilon_f=0.01, n_epsilon=0.1):
        self.epsilon_i = epsilon_i
        self.epsilon_f = epsilon_f
        self.n_epsilon = n_epsilon
        self.epsilon = epsilon_i
        self.gamma = gamma
        self.state_shape = state_shape
        self.num_actions = num_actions
        self.optimizer = tf.keras.optimizers.Adam(alpha) 

        self.Q = ConvNN(state_shape,num_actions).build()
        self.Q_ = ConvNN(state_shape,num_actions).build()
        
    def synchronize(self):
        self.Q_.set_weights(self.Q.get_weights())

    def act(self, s_t):
        if np.random.random() < self.epsilon:
            return np.random.randint(self.num_actions,size=3)
        return np.argmax(self.Q(s_t), axis=1)
    
    def decay_epsilon(self, n):
        self.epsilon = max(
            self.epsilon_f, 
            self.epsilon_i - (n/self.n_epsilon)*(self.epsilon_i - self.epsilon_f))

    def update(self, s_t, a_t, r_t, s_t_next, d_t):
        with tf.GradientTape() as tape:
            Q_next = tf.stop_gradient(tf.reduce_max(self.Q_(s_t_next), axis=1))
            Q_pred = tf.reduce_sum(self.Q(s_t)*tf.one_hot(a_t, self.num_actions, dtype=tf.float32), axis=1)
            loss = tf.reduce_mean(0.5*(r_t + (1-d_t)*self.gamma*Q_next - Q_pred)**2)
        grads = tape.gradient(loss, self.Q.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.Q.trainable_variables))

The VectorizedEnvWrapper allows us to use multiple envs in parallel

In [ ]:
class VectorizedEnvWrapper(gym.Wrapper):
    def __init__(self, make_env, num_envs=1):
        super().__init__(make_env())
        self.num_envs = num_envs
        self.envs = [make_env() for env_index in range(num_envs)]
    
    def reset(self):
        return np.asarray([env.reset() for env in self.envs])
    
    def reset_at(self, env_index):
        return self.envs[env_index].reset()
    
    def step(self, actions):
        next_states, rewards, dones, infos = [], [], [], []
        for env, action in zip(self.envs, actions):
            next_state, reward, done, info = env.step(action)
            next_states.append(next_state)
            rewards.append(reward)
            dones.append(done)
            infos.append(info)
        return np.asarray(next_states), np.asarray(rewards), \
            np.asarray(dones), np.asarray(infos)

Just dividing by 255 to get tensors filled with 0 and 1

In [ ]:
class NormalizationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def observation(self, obs):
        return obs/255

We produce a train_agent and the Gfootball env

In [ ]:
train_agent = Agent((72,96,4),19,alpha=0.1,gamma=0.95)
train_agent.Q.save_weights("QWeights")
train_agent.Q_.save_weights("Q_Weights")
smm_env = VectorizedEnvWrapper(lambda: NormalizationWrapper(gym.make("GFootball-11_vs_11_kaggle-SMM-v0")),1)
train_buffer = ReplayBuffer()

The training method is defined below 

In [ ]:
def train(env=smm_env,T=3001,batch_size=3,sync_every=10,agent=train_agent,buffer=train_buffer):
    
    agent.Q.load_weights("QWeights")
    agent.Q_.load_weights("Q_Weights")
    rewards = []
    episode_rewards = 0
    s_t = env.reset()
    
    for t in range(T):
        if t%sync_every == 0:
            agent.synchronize()
        a_t = agent.act(s_t)
        s_t_next, r_t, d_t, info = env.step(a_t)
        buffer.remember(s_t, a_t, r_t, s_t_next, d_t)
        for batch in buffer.sample(batch_size):
            agent.update(*batch)
        agent.decay_epsilon(t/T)
        episode_rewards += r_t
           
    for i in range(env.num_envs):
        rewards.append(episode_rewards[i])
        episode_rewards[i] = 0
        s_t[i] = env.reset_at(i)
        
    agent.epsilon_i = 1.0
    agent.epsilon_f = 0.01
    agent.n_epsilon = 0.1
    agent.epsilon = agent.epsilon_i
    
    return rewards

We train the model a couple of times

In [ ]:
for i in range(2):
    print(train())
    train_agent.Q.save_weights("QWeights")
    train_agent.Q_.save_weights("Q_Weights")

To write a submission, we only need a lightened version of the agent, the ConvNN architecture, and to load the trained weights


In [ ]:
%%writefile submission.py

from gfootball.env import observation_preprocessing
import numpy as np
import tensorflow as tf

class SplitLayer(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def call(self, inputs):
        split0,split1,split2,split3 = tf.split(inputs,4,3)
        return [split0,split1,split2,split3]
    
class ConvNN:

    def __init__(self,state_shape,num_actions):
        self.state_shape = state_shape
        self.num_actions = num_actions

    def _model_architecture(self):
    
        frames_input = tf.keras.layers.Input(shape=self.state_shape,name='input',batch_size=1)
        frames_split = SplitLayer()(frames_input)
        conv_branches = []
        for f,frame in enumerate(frames_split):
            conv_branches.append(self._build_conv_branch(frame,f))

        concat = tf.keras.layers.concatenate(conv_branches)
        
        fc0 = tf.keras.layers.Dense(units=8192, name='fc0', 
                                     activation='relu')(concat)
        fc1 = tf.keras.layers.Dense(units=2048, name='fc1', 
                                     activation='relu')(fc0)
        fc2 = tf.keras.layers.Dense(units=512, name='fc2', 
                                     activation='relu')(fc1)
        fc3 = tf.keras.layers.Dense(units=256, name='fc3', 
                                     activation='relu')(fc2)
        fc4 = tf.keras.layers.Dense(units=64, name='fc4', 
                                     activation='relu')(fc3)
        
        action_output = tf.keras.layers.Dense(units=self.num_actions, name='output',
                                               activation='relu')(fc4)
        return frames_input,action_output
        
    @staticmethod
    def _build_conv_branch(frame,number):
        conv1 = tf.keras.layers.Conv2D(16, kernel_size=(8, 8), strides=(4, 4),
                                        name='conv1_frame'+str(number), padding='same',
                                        activation='relu')(frame)
        mp1 = tf.keras.layers.MaxPooling2D(pool_size=2, name="mp1_frame"+str(number))(conv1)
        conv2 = tf.keras.layers.Conv2D(24, kernel_size=(4, 4), strides=(2, 2),
                                        name='conv2_frame'+str(number),padding='same',
                                        activation='relu')(mp1)
        mp2 = tf.keras.layers.MaxPooling2D(pool_size=2, name="mp2_frame"+str(number))(conv2)
        conv3 = tf.keras.layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1),
                                        name='conv3_frame'+str(number),padding='same',
                                        activation='relu')(mp2)
        mp3 = tf.keras.layers.MaxPooling2D(pool_size=2, name="mp3_frame"+str(number))(conv3)
        conv4 = tf.keras.layers.Conv2D(64, kernel_size=(2, 2), strides=(1, 1),
                                        name='conv4_frame'+str(number), padding='same',
                                        activation='relu')(mp3)
        mp4 = tf.keras.layers.MaxPooling2D(pool_size=1, name="mp4_frame"+str(number))(conv4)

        flatten = tf.keras.layers.Flatten(name='flatten'+str(number))(mp4)

        return flatten
        
    def build(self):
        frames_input,action_output = self._model_architecture()
        model = tf.keras.Model(inputs=[frames_input], outputs=[action_output])
        return model

class Agent:
    def __init__(self, state_shape, num_actions):
        
        self.state_shape = state_shape
        self.num_actions = num_actions

        self.Q = ConvNN(state_shape,num_actions).build()
        self.Q_ = ConvNN(state_shape,num_actions).build()
        
    def act(self, s_t):
        return np.argmax(self.Q(s_t), axis=1)
    
DQN_Agent = Agent((72,96,4),19)

DQN_Agent.Q.load_weights("QWeights")
DQN_Agent.Q_.load_weights("Q_Weights")

def agent(obs):
    obs = obs['players_raw'][0]
    obs = observation_preprocessing.generate_smm([obs])
    obs = obs/255 #Normalization
    action = DQN_Agent.act(obs)
    return [action[0]]

We can see how our agent perform against a 'do_nothing' opponent

In [ ]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/submission.py", "do_nothing"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)

The agent makes very poor decisions, surely because it hasn't trained enough ([Google Research Paper on Gfootball](https://arxiv.org/pdf/1907.11180.pdf) implies that we need a lot more of training). But I think there are different aspects we can still improve on in order to get slightly better results. Maybe a [CheckpointRewardWrapper](https://github.com/google-research/football/blob/7a5ae3b5c849beb69d15a7c0028f16d7297b1d9f/gfootball/env/wrappers.py#L275) adapted for SMM representations returning a more explicit reward to the agent could help